In [9]:
from quilt.data.haradai1262 import YouTuber

In [10]:
#データの確認
YouTuber.channels.UUUM().head()

,id,title,description,publishedAt,thumbnails,viewCount,subscriberCount,hiddenSubscriberCount,videoCount,commentCount
0,UCZf__ehlCEBPop___sldpBUQ,HikakinTV,HikakinTVはヒカキンが日常の面白いものを紹介するチャンネルです。\r\r\r\n\r...,2011-07-19T11:31:43.000Z,https://yt3.ggpht.com/-NFhw6-eus8Y/AAAAAAAAAAI...,4862671898,6173850,False,2467,0
1,UCgMPP6RRjktV7krOfyUewqw,はじめしゃちょー（hajime）,動画を出したい時にテキトーにアップロードします。\r\r\r\nオールジャンル!!!!なんで...,2012-08-31T04:19:50.000Z,https://yt3.ggpht.com/-zuf1TvgF3Ik/AAAAAAAAAAI...,5109135195,6439601,False,1661,0
2,UCg4nOl7_gtStrLwF0_xoV0A,SeikinTV,SEIKIN（セイキン）／ YouTuber（ユーチューバー）\r\r\r\nYouTube...,2012-01-29T16:53:15.000Z,https://yt3.ggpht.com/-TpVHBaXKoW0/AAAAAAAAAAI...,2437647850,3132078,False,1535,0
3,UCFBjsYvwX7kWUjQoW7GcJ5A,瀬戸弘司 / Koji Seto,こんにちは。劇団天然工房の瀬戸弘司です。 \r\r\r\n役者として舞台に立ったり、脚本を書...,2010-04-10T02:11:44.000Z,https://yt3.ggpht.com/-Ox6vE1qB9g4/AAAAAAAAAAI...,714179677,1449451,False,1796,0
4,UCzWygEC8zQKYpCp7nwX_M__A,PDSKabushikiGaisha,本名・ペインダンテ将之介\r\r\r\n\r\r\r\n\r\r\r\n20才のイギリスとア...,2011-01-06T23:32:56.000Z,https://yt3.ggpht.com/-aQrjxurrGL4/AAAAAAAAAAI...,1012465601,1419278,False,2236,0


In [11]:
# HIKAKINの動画のみを抽出します。
df_bigi = YouTuber.channel_videos.UUUM_videos()
df_bigi = df_bigi[df_bigi['cid'] == 'UCZf__ehlCEBPop___sldpBUQ'].reset_index(drop=True)
df = df_bigi.loc[:,["title","tags"]] 

In [12]:
# 映像に含まれるタイトルやタグを基にクラスタリングをします。
# タイトルから名詞を抽出し、タグと結合し、コーパスを作成します。

from janome.tokenizer import Tokenizer
tokenizer = Tokenizer()

def get_noun( str ):
    nouns = []
    for token in tokenizer.tokenize( str ):
        pos = token.part_of_speech.split(',')[0]
        if pos == '名詞': nouns.append( token.base_form )
    return nouns

def tag2list( str ):
    tags = []
    for i in str[1:-1].split(','):
        i = i.replace(' ','')
        tags.append( i[1:-1] )
    return tags

df['title_noun'] = df['title'].apply( get_noun )
df['tags_noun'] = df['tags'].apply( tag2list )
X_tmp = df.loc[:,['title_noun','tags_noun']].values
X = [ i[0] + i[1] for i in X_tmp ]

In [13]:
# 作成したコーパスにLSIを適用し、各映像の特徴量を抽出します。
# LSIには、gensimを使用しており、"dictionary.filter_extremes(no_below=2, no_above=0.5 )"で語彙を調整しています。

import gensim
from gensim import corpora, models, similarities
import numpy as np

dictionary = corpora.Dictionary( X )
dictionary.filter_extremes(no_below=2, no_above=0.5 )
corpus = [dictionary.doc2bow(text) for text in X]
lsi = models.LsiModel(corpus, id2word=dictionary, num_topics=30)
lsi_feature = lsi[corpus]

X_lsi = np.ndarray(( len(corpus), 30 ))
for i in range( len( corpus ) ):
    for j in lsi_feature[i]:
        X_lsi[i, j[0]] = j[1]

In [14]:
# 得られた特徴量を用いてクラスタリング(今回はK-means法、クラスタ数 20)を適用します。

def get_groups( t_cluster ):
    groups_tmp = {}
    for i, l in enumerate( t_cluster ):
        if not l in groups_tmp: groups_tmp[l] = []
        groups_tmp[l].append( i )
    return groups_tmp

from sklearn.cluster import KMeans
cluster = KMeans(n_clusters=15, random_state=0).fit(X_lsi)
group = get_groups( cluster.labels_ )

In [15]:
import pandas as pd
def get_cluster_means( x_tmp, group_tmp ):
    mean_df_tmp= x_tmp.mean()
    mean_df_tmp.name = -1
    for i in range( len( group_tmp ) ):
        c = pd.DataFrame(x_tmp,index=group_tmp[i])
        c_mean = c.mean()
        c_mean.name = i 
        mean_df_tmp = pd.concat([mean_df_tmp, c_mean], axis=1)
    mean_df_tmp = mean_df_tmp.drop( columns=[-1] )
    return mean_df_tmp

df_stat = df_bigi.loc[:,['viewCount','dislikeCount']]
mean_df = get_cluster_means( df_stat, group ).T
#mean_df['like/dislike'] = mean_df['likeCount'] / mean_df['dislikeCount']
mean_df.style.background_gradient( cmap='Oranges' )

,viewCount,dislikeCount
0,3.13781e+06,1735.33
1,4.08348e+06,2974.29
2,3.77141e+06,3818.2
3,2.40266e+06,2021.9
4,5.7926e+06,4477.72
5,1.37756e+06,992.933
6,3.09617e+06,1912.44
7,2.46526e+06,1697.36
8,4.57079e+06,3850.15
9,6.55301e+06,4968.75


## 得られたクラスター
### cluster: 0
    379  アブラたりてます!? EDGE驚き&やりすぎ鬼背脂とんこつ醤油ラーメン食べてみた！
    403  カップ麺にも激辛スパイス入れすぎてみた - EDGE 鬼辛とんこつ醤油ラーメン
    408  【激辛】蒙古タンメン中本の『北極ラーメン』のカップ麺食べてみた！
    411  激辛焼そばJACK食べてみた！
    415  EDGE - カップ焼そばに激辛スパイス入れすぎた件 食べてみた！
### cluster: 1
    159  YouTubeテーマソング -Tetsuya Komuro Rearrange-
    160  【1つだけ超激辛】フィッシャーズ＆ヒカキンでロシアンチョコレートやったら爆笑しすぎたw
    169  ゲーセンのクレーンゲームでお菓子とりまくってやんよ！
    178  ヒカキン怒る！ピカチュウバッテリーの転売価格にブチ切れ【ポケモン】
    181  復刻版ポケモンカード箱買いしたら大当たり!?
### cluster: 2
    0  【大食い】超高級寿司店で３人で食べ放題したらいくらかかるの!?【大トロ1カン2,000円】
    1  【女王集結】女性YouTuberたちと飲みながら本音トークしてみたら爆笑www
    2  【悪質】偽物ヒカキン許さねぇ…注意してください！【なりすまし】
    4  【放送事故】酒飲みながら東海オンエア×ヒカキンで質問コーナーやったらヤバかったwww
    7  【3000万円超え!?】ヒカキン、人生最大の買い物！ルイ・ヴィトン×FIFA W杯コラボ商品全部買ってみた！【2018 ロシアW杯・2018 FIFA World Cup Russia】
### cluster: 3
    15  チビキンちゃんにドン・キホーテから謎のダンボールが届いたのでプレゼントしてみた！
    34  パンじゃなくてトマトではさんであるハンバーガーがヤベェw【とま実バーガー】【モスバーガー】
    173  PS Vitaマインクラフトコラボモデルがｷﾀ━━(ﾟ∀ﾟ)━━!!!!
    190  街でYouTuberのイメージ聞いてみた！ヒカキンTVインタビュー【原宿編】
    218  ゴジラのたまご食べてみた！
### cluster: 4
    312  YouTubeテーマソングレコーディング風景！
    321  今年もYouTuberイベント開催！&ボンボンTVスタート！
    322  【負傷】リフティング100回目指してたら悲劇が…
    324  卓上パンチングマシン本気で殴りまくってみた！Stress Buster
    329  ワンダーコア TVCM パロディ ヒカキン編
### cluster: 5
    354  【プレゼント企画】ヒカキンTV200万人突破記念にiPod touchプレゼント！
    431  プリクラアプリで詐欺写メ撮ってみた！BeautyPlus
    443  期間限定！パピコバナナオーレが美味い！
    445  ヒカキンタンブラー10名様にプレゼント！スマポ×HikakinTV！
    446  一平ちゃん焼そばコーンポタージュ味食べてみた！
### cluster: 6
    305  YouTuber人生振り返ってみた。#YouTuberはじめました
    306  【ヒカキン】1本目の動画 60秒ver.【My First YouTube】
    307  フルート × ビートボックス！初めてフルート吹いてみた！
    308  【世界最大!?】超巨大ヘビグミ食べてみた！
    309  大量のドライアイスを使って隠れ身の術してみた！
### cluster: 7
    6  しまむらでセイキンが働いてたwww ヒカキン&セイキンしまむらでグッズ発売決定！
    56  Mステ２時間スペシャル出演決定の喜び
    75  【大暴走】雪にダイブして雪食べまくるwww【恒例行事2018年ver.】
    78  【裏話】Mステの裏がヤバすぎたので話します！【Mステスーパーライブ2017】
    79  Mステの裏側を撮影して来ましたw
### cluster: 8
    317  好きな曲熱唱してみた！ヒカキンオススメ曲紹介！【AWA】
    318  火が消えないロウソクの火を全力で消してみた！『マジックキャンドル』
    320  【閲覧注意】鼻毛一気に全部抜いてみた！鼻毛脱毛ブラジリアンワックス『GOSSO』
    323  液体磁石で遊んでみたらすごいことになった！
    325  ユーチューバーで『パピプペパピコゲーム』やってみた！
### cluster: 9
    74  400万の純金ハンドスピナー分解してみたw【金の延べ棒】【$40,000  24K Fidget Spinner 】
    77  400万円の純金ハンドスピナー買ったらヤバすぎたwww
    124  【最強決定】ハンドスピナー頂上決戦！民族 vs ニセ民族！【ヒカキン vs セイキン】【Fidget Spinner】
    138  ハンドスピナーどれが一番長く回るか選手権！ヒカキン vs セイキン！【Fidget Spinner】
### cluster: 10
    3  ツイッターのヒカキンシンメトリーBotが面白すぎて爆笑www
    5  W杯後の渋谷でゴミ拾いしたらゴミの量ヤバかった…【ロシアW杯 日本 vs コロンビア戦】
    12  【世界に10個限定】10kgのダンベルiPhoneケースがもはやケースじゃないwww【ジャパネットひかた】
    14  【壮絶】激レアなユニクロドラえもんぬいぐるみ争奪戦 in 横浜【村上隆さんコラボ商品】
    16  ユーチューバースポーツ王決定戦！運動神経が1番いいのは誰だ!?【ヒカキン vs ンダホ vs てつや】
### cluster: 11
    261  ペッパーピアノ弾いてみた！【pepper】
    265  罰ゲームかけてポカポンゲームやってみた！これオモロイw
    292  【閲覧注意】毛穴の汚れ吸引してみた！パナソニック スポットクリア！
    298  Pepper(ペッパー)がやってきた！人型ロボット！
### cluster: 12
    158  ヒカキン家の玄関がマジでヤバイことに…
    161  【暴露】ヒカキンの質問コーナー #ぶっちゃけヒカキン 回答編！【ギリギリ発言】
    162  ついにやっちまった…
    163  【悲報】スタバのキーホルダーが欲しくてのたうち回る27歳男性の悲劇に全米が泣いた
    164  アップルの初売りで買ったものがついに届いたぜ！
### cluster: 13
    19  【50,000円分】メルカリでアルミ玉を大量購入したら超ヤバいのあったwww【アルミホイル・鉄球】【ボール】
    21  【超巨大】アルミホイル250mハンマーで叩きまくったらピカピカ巨大鉄球出来たwww【アルミホイル玉】【ボール】
### cluster: 14
    17  メルカリで買ったアルミ玉磨いたら超ピカピカにwww【劇的ビフォーアフター】
    42  【もはや鏡】アルミホイル2日間ハンマーで叩いたら超ピカピカの鉄球出来たw【ボール】
    60  カップヌードルポテナゲにガチのポテナゲ入れて最強にして食べてみたwww
    117  【プレゼント企画】ニンテンドースイッチ スプラトゥーン２セット10台10名様にプレゼント！

In [38]:
# 実行ごとにクラスターの順番が前後するので確認用
cluster_number = 14
group[cluster_number]

[17, 42, 60, 117]